 # Food Classification Using Deep Neural Networks and Transfer Learning

# 1. Data Splitting 

Split the data into training, validation, and test sets. Justify your choice.

In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

import matplotlib.pyplot as plt


In [2]:
#move to gpu if possible

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
print("using device", device)

using device cuda


# Load Data

In [3]:
def get_data_loader(target_classes, batch_size):
    """ Returns the indices for datapoints in the dataset that
    belongs to the desired target classes, a subset of all possible classes.

    Args:
        dataset: Dataset object
        classes: A list of strings denoting the name of each class
        target_classes: A list of strings denoting the name of the desired
                        classes. Should be a subset of the argument 'classes'
    Returns:
        indices: list of indices that have labels corresponding to one of the
                 target classes
    """
    classes = ('falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels','chicken_wings','chocolate_cake','fried_rice','greek_salad','hamburger','lasagna','poutine','spring_rolls','waffles','spaghetti_carbonara')


    from torchvision import transforms

    # Set up a transform that scales and crops an image so it has the dimensions
    # of the input layer of alexnet
    scale_crop = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224)
    ])
  
    # The normalization that was applied to the data when alexnet was trained
    normalize = transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
    )

    # Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
    preprocess = transforms.Compose([
      scale_crop,
      transforms.ToTensor(),
      normalize
    ])
    trainset = torchvision.datasets.ImageFolder(root = 'train_more_food',
                                                 transform=preprocess)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='val_more_food',
                                                 transform=preprocess)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'test_more_food',
                                            transform=preprocess)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)

    return train_loader, val_loader, test_loader, classes

In [4]:
    from torchvision import transforms

    # Set up a transform that scales and crops an image so it has the dimensions
    # of the input layer of alexnet
    scale_crop = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224)
    ])
  
    # The normalization that was applied to the data when alexnet was trained
    normalize = transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
    )

    # Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
    preprocess = transforms.Compose([
      scale_crop,
      transforms.ToTensor(),
      normalize
    ])
    trainset = torchvision.datasets.ImageFolder(root = 'ig/train_more_food',
                                                 transform=preprocess)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = 128,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='ig/val_more_food',
                                                 transform=preprocess)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=128,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'ig/test_more_food',
                                            transform=preprocess)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=128,
                                             num_workers=1, shuffle = True)

In [6]:
trainset.class_to_idx


{'apple_pie': 0,
 'chicken_wings': 1,
 'chocolate_cake': 2,
 'donuts': 3,
 'falafel': 4,
 'french_fries': 5,
 'fried_rice': 6,
 'greek_salad': 7,
 'hamburger': 8,
 'lasagna': 9,
 'macarons': 10,
 'mussels': 11,
 'nachos': 12,
 'onion_rings': 13,
 'oysters': 14,
 'pizza': 15,
 'poutine': 16,
 'spaghetti_carbonara': 17,
 'spring_rolls': 18,
 'waffles': 19}

In [11]:
def evaluate(net, loader, criterion):
    """ Evaluate the network on the validation set.

     Args:
         net: PyTorch neural network object
         loader: PyTorch data loader for the validation set
         criterion: The loss function
     Returns:
         err: A scalar for the avg classification error over the validation set
         loss: A scalar for the average loss function over the validation set
     """
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0

    for i, data in enumerate(loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        total_err += pred.ne(labels.view_as(pred)).sum().item()
        total_loss += loss.item()
        total_epoch += len(labels)

    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)

    return err, loss

def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

def trainAlexNet(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
   
    
    from torchvision import transforms

    # Set up a transform that scales and crops an image so it has the dimensions
    # of the input layer of alexnet
    scale_crop = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224)
    ])
  
    # The normalization that was applied to the data when alexnet was trained
    normalize = transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
    )

    # Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
    preprocess = transforms.Compose([
      scale_crop,
      transforms.ToTensor(),
      normalize
    ])
    #trainset = torchvision.datasets.ImageFolder(augmented_train_data,
      #                                           transform=preprocess)
 
    train_loader = torch.utils.data.DataLoader(train_with_aug, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='ig/val_more_food',
                                                 transform=preprocess)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'ig/test_more_food',
                                            transform=preprocess)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)
    
    
    torch.manual_seed(1000)

   
  
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
      
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
           
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
                      
            optimizer.zero_grad()
            outputs = net(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
           
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name("RESNET", batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)
        

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
    
# Importing relevant Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import os
import torchvision
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms
from torch.nn import functional as F
import copy
from torch.autograd import Variable


###############################################################################
# Feature Extraction using AlexNet pretrained model

class AlexNetDA(nn.Module):
    '''
    Class that loads AlexNet Feature Model ('Convolution layers') with imagenet trained weights
    
    input : image tensors with dimension Lx3x224x224
    
    output : feature tensor with dimension Lx256x6x6
    
    *L - Batch size
    
    '''
    
    def load_weights(self):
        an_builtin = torchvision.models.alexnet(pretrained=True) # Loads the pretrained model weights
        
        features_weight_i = [0, 3, 6, 8, 10]
        for i in features_weight_i:
            self.features[i].weight = an_builtin.features[i].weight
            self.features[i].bias = an_builtin.features[i].bias

    def __init__(self):
        super(AlexNetDA, self).__init__()
        self.name = "AlexNetDA"
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 20),
        )
        self.load_weights() # Copies the weights to AlexNetFeatures model layers

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        
        x = self.classifier(x)
        return x
        
        

        
          

In [28]:
netalex = AlexNetDA()
#netalex.to(device)
device_ids = [i for i in range(torch.cuda.device_count())]
print(device_ids)
model = nn.DataParallel(netalex, [0,1])
model = model.to(device)
trainAlexNet(model, batch_size = 32, learning_rate = 0.00001, num_epochs = 20)

[0, 1]
Epoch 1: Train err: 0.3785833333333333, Train loss: 1.2762629676659902  |Validation err: 0.27075 , Validation loss:0.911521057844162 
Epoch 2: Train err: 0.23333333333333334, Train loss: 0.7891806231737137  |Validation err: 0.266 , Validation loss:0.876939565896988 
Epoch 3: Train err: 0.17825, Train loss: 0.6175516118804614  |Validation err: 0.25725 , Validation loss:0.8859039123058319 
Epoch 4: Train err: 0.145375, Train loss: 0.5075232909321785  |Validation err: 0.247 , Validation loss:0.8823808536529542 
Epoch 5: Train err: 0.12995833333333334, Train loss: 0.4593242875734965  |Validation err: 0.24275 , Validation loss:0.8407653880119323 
Epoch 6: Train err: 0.12283333333333334, Train loss: 0.4233424866100152  |Validation err: 0.24 , Validation loss:0.8589568246603012 
Epoch 7: Train err: 0.111125, Train loss: 0.3873286395271619  |Validation err: 0.231 , Validation loss:0.8205231423377991 
Epoch 8: Train err: 0.10595833333333333, Train loss: 0.37267463718851407  |Validation e

In [29]:
netalex = AlexNetDA()
#netalex.to(device)
device_ids = [i for i in range(torch.cuda.device_count())]
print(device_ids)
model = nn.DataParallel(netalex, [0,1])
model = model.to(device)
trainAlexNet(model, batch_size = 128, learning_rate = 0.001, num_epochs = 20)

[0, 1]
Epoch 1: Train err: 0.9529166666666666, Train loss: 3.0282405437307154  |Validation err: 0.95 , Validation loss:2.9957598969340324 
Epoch 2: Train err: 0.9520833333333333, Train loss: 2.9959838529850575  |Validation err: 0.95 , Validation loss:2.9957447201013565 
Epoch 3: Train err: 0.9522083333333333, Train loss: 2.9959474954199283  |Validation err: 0.95 , Validation loss:2.9957177191972733 
Epoch 4: Train err: 0.95125, Train loss: 2.995912674893724  |Validation err: 0.95 , Validation loss:2.995738409459591 
Epoch 5: Train err: 0.9526666666666667, Train loss: 2.9958997825358775  |Validation err: 0.95 , Validation loss:2.995729185640812 
Epoch 6: Train err: 0.952875, Train loss: 2.9959137744091926  |Validation err: 0.95 , Validation loss:2.9957431107759476 
Epoch 7: Train err: 0.9545, Train loss: 2.9959080650451337  |Validation err: 0.95 , Validation loss:2.9957460686564445 
Epoch 8: Train err: 0.9534166666666667, Train loss: 2.9959142956327884  |Validation err: 0.95 , Validatio

In [30]:
netalex = AlexNetDA()
#netalex.to(device)
device_ids = [i for i in range(torch.cuda.device_count())]
print(device_ids)
model = nn.DataParallel(netalex, [0,1])
model = model.to(device)
trainAlexNet(model, batch_size = 128, learning_rate = 0.0001, num_epochs = 20)

[0, 1]
Epoch 1: Train err: 0.3459583333333333, Train loss: 1.1686042778669519  |Validation err: 0.262 , Validation loss:0.8850515745580196 
Epoch 2: Train err: 0.19420833333333334, Train loss: 0.6483240877377227  |Validation err: 0.264 , Validation loss:0.9105774108320475 
Epoch 3: Train err: 0.14154166666666668, Train loss: 0.48045425735255504  |Validation err: 0.25025 , Validation loss:0.9179508239030838 
Epoch 4: Train err: 0.12366666666666666, Train loss: 0.4107015075043161  |Validation err: 0.23875 , Validation loss:0.899783294647932 
Epoch 5: Train err: 0.11058333333333334, Train loss: 0.3735055167307245  |Validation err: 0.2395 , Validation loss:0.8464423958212137 
Epoch 6: Train err: 0.105625, Train loss: 0.3566283643087174  |Validation err: 0.23 , Validation loss:0.8648484852164984 
Epoch 7: Train err: 0.09529166666666666, Train loss: 0.3241417207457918  |Validation err: 0.21925 , Validation loss:0.8630065470933914 
Epoch 8: Train err: 0.09245833333333334, Train loss: 0.302666

In [31]:
netalex = AlexNetDA()
#netalex.to(device)
device_ids = [i for i in range(torch.cuda.device_count())]
print(device_ids)
model = nn.DataParallel(netalex, [0,1])
model = model.to(device)
trainAlexNet(model, batch_size = 64, learning_rate = 0.0001, num_epochs = 20)

[0, 1]
Epoch 1: Train err: 0.34579166666666666, Train loss: 1.1604937566121418  |Validation err: 0.2565 , Validation loss:0.8742000753917392 
Epoch 2: Train err: 0.18991666666666668, Train loss: 0.6340108564694722  |Validation err: 0.24075 , Validation loss:0.8327392888447595 
Epoch 3: Train err: 0.14370833333333333, Train loss: 0.48019088558355966  |Validation err: 0.25725 , Validation loss:0.9308844161412072 
Epoch 4: Train err: 0.12458333333333334, Train loss: 0.4142082628806432  |Validation err: 0.251 , Validation loss:0.9071697062916226 
Epoch 5: Train err: 0.11254166666666666, Train loss: 0.3800031027793884  |Validation err: 0.227 , Validation loss:0.826458605035903 
Epoch 6: Train err: 0.10445833333333333, Train loss: 0.3532179073095322  |Validation err: 0.22975 , Validation loss:0.8759564076151166 
Epoch 7: Train err: 0.09841666666666667, Train loss: 0.3292626137733459  |Validation err: 0.2265 , Validation loss:0.8822438172877781 
Epoch 8: Train err: 0.09316666666666666, Train 

In [32]:
netalex = AlexNetDA()
#netalex.to(device)
device_ids = [i for i in range(torch.cuda.device_count())]
print(device_ids)
model = nn.DataParallel(netalex, [0,1])
model = model.to(device)
trainAlexNet(model, batch_size = 64, learning_rate = 0.00001, num_epochs = 20)

[0, 1]
Epoch 1: Train err: 0.3983333333333333, Train loss: 1.3596372736295064  |Validation err: 0.28725 , Validation loss:0.9695532984203763 
Epoch 2: Train err: 0.254875, Train loss: 0.8571073141098022  |Validation err: 0.27175 , Validation loss:0.9154061495311676 
Epoch 3: Train err: 0.19854166666666667, Train loss: 0.6813607420921326  |Validation err: 0.27325 , Validation loss:0.9202023233686175 
Epoch 4: Train err: 0.16004166666666667, Train loss: 0.5658844091892242  |Validation err: 0.2485 , Validation loss:0.8817730810907152 
Epoch 5: Train err: 0.13945833333333332, Train loss: 0.49756006654103596  |Validation err: 0.24875 , Validation loss:0.8621167857495565 
Epoch 6: Train err: 0.1295, Train loss: 0.4589031814734141  |Validation err: 0.2465 , Validation loss:0.8545234070883857 
Epoch 7: Train err: 0.116625, Train loss: 0.4210182774861654  |Validation err: 0.235 , Validation loss:0.8286143531875004 
Epoch 8: Train err: 0.11529166666666667, Train loss: 0.4024809059302012  |Valida

# 2. Convolutional Network - Baseline Model 
Build a convolutional neural network model that takes the (224x224 RGB) image as input.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
 
class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# convolutional neural network, 
class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 5, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 3, padding = 1)
        self.conv3 = nn.Conv2d(10, 15, 3, padding = 1)
        self.fc1 = nn.Linear(15 * 28 * 28 , 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 15 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

# 3. Training 
Train your network. Plot the training curve.

Make sure that you are checkpointing frequently!

In [6]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path


In [7]:
def evaluate(net, loader, criterion):
    """ Evaluate the network on the validation set.

     Args:
         net: PyTorch neural network object
         loader: PyTorch data loader for the validation set
         criterion: The loss function
     Returns:
         err: A scalar for the avg classification error over the validation set
         loss: A scalar for the average loss function over the validation set
     """
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0

    for i, data in enumerate(loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        total_err += pred.ne(labels.view_as(pred)).sum().item()
        total_loss += loss.item()
        total_epoch += len(labels)

    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)

    return err, loss

In [12]:
def trainResNet(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
   
    
    from torchvision import transforms

    # Set up a transform that scales and crops an image so it has the dimensions
    # of the input layer of alexnet
    scale_crop = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224)
    ])
  
    # The normalization that was applied to the data when alexnet was trained
    normalize = transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
    )

    # Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
    preprocess = transforms.Compose([
      scale_crop,
      transforms.ToTensor(),
      normalize
    ])
    trainset = torchvision.datasets.ImageFolder(root = 'ig/train_more_food',
                                                 transform=preprocess)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='ig/val_more_food',
                                                 transform=preprocess)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'ig/test_more_food',
                                            transform=preprocess)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)
    
    
    torch.manual_seed(1000)

   
  
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.fc.parameters(), lr=learning_rate, weight_decay=1e-5)
    #optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
      
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
           
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
                      
            optimizer.zero_grad()
            outputs = net(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
           
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name("RESNET", batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)
        end_time = time.time()
        elapsed_time = end_time - start_time

        print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
        
          

In [18]:
def trainBaseline(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
    from torchvision import transforms

    # Set up a transform that scales and crops an image so it has the dimensions
    # of the input layer of alexnet
    scale_crop = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224)
    ])
  
    # The normalization that was applied to the data when alexnet was trained
    normalize = transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
    )

    # Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
    preprocess = transforms.Compose([
      scale_crop,
      transforms.ToTensor(),
      normalize
    ])
    trainset = torchvision.datasets.ImageFolder(root = 'train_more_food',
                                                 transform=preprocess)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='val_more_food',
                                                 transform=preprocess)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'test_more_food',
                                            transform=preprocess)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)
    
    torch.manual_seed(1000)

   
  
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
      
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
           
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
                      
            optimizer.zero_grad()
            outputs = net(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
           
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))
        
        # Save the current model (checkpoint) to a file
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)
        print("Total time elapsed: {:.1f} seconds".format(elapsed_time))

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
        
          

In [8]:
# Training Curve
import matplotlib.pyplot as plt
def plot_training_curve(path):
    """ Plots the training curve for a model run, given the csv files
    containing the train/validation error/loss.

    Args:
        path: The base path of the csv files produced during training
    """

    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    plt.plot(range(1,16), train_err, label="Train")
    plt.plot(range(1,16), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,16), train_loss, label="Train")
    plt.plot(range(1,16), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

# Test Accuracy

In [44]:
# Importing relevant Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import os
import torchvision
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms
from torch.nn import functional as F
import copy
from torch.autograd import Variable


###############################################################################
# Feature Extraction using AlexNet pretrained model

class AlexNet(nn.Module):
    '''
    Class that loads AlexNet Feature Model ('Convolution layers') with imagenet trained weights
    
    input : image tensors with dimension Lx3x224x224
    
    output : feature tensor with dimension Lx256x6x6
    
    *L - Batch size
    
    '''
    
    def load_weights(self):
        an_builtin = torchvision.models.alexnet(pretrained=True) # Loads the pretrained model weights
        
        features_weight_i = [0, 3, 6, 8, 10]
        for i in features_weight_i:
            self.features[i].weight = an_builtin.features[i].weight
            self.features[i].bias = an_builtin.features[i].bias

    def __init__(self):
        super(AlexNet, self).__init__()
        self.name = "AlexNet"
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 20),
        )
        self.load_weights() # Copies the weights to AlexNetFeatures model layers

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
        


In [54]:
def trainAlexNet(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
   
    
    from torchvision import transforms

    # Set up a transform that scales and crops an image so it has the dimensions
    # of the input layer of alexnet
    scale_crop = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224)
    ])
  
    # The normalization that was applied to the data when alexnet was trained
    normalize = transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
    )

    # Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
    preprocess = transforms.Compose([
      scale_crop,
      transforms.ToTensor(),
      normalize
    ])
    trainset = torchvision.datasets.ImageFolder(root = 'ig/train_more_food',
                                                 transform=preprocess)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='ig/val_more_food',
                                                 transform=preprocess)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'ig/test_more_food',
                                            transform=preprocess)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)
    
    
    torch.manual_seed(1000)

   
  
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
      
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
           
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
                      
            optimizer.zero_grad()
            outputs = net(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name("RESNET", batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)
        end_time = time.time()
        elapsed_time = end_time - start_time

        print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
        
          

In [51]:
model = AlexNet()
#netalex.to(device)

#model = nn.DataParallel(netalex, [0,1])
model = model.to(device)

In [ ]:
trainAlexNet(model, batch_size = 64, learning_rate = 0.00001, num_epochs = 15)

Epoch 1: Train err: 0.648, Train loss: 2.1855580090208258  |Validation err: 0.4725 , Validation loss:1.5331826739841037 
Total time elapsed: 91.61 seconds
Epoch 2: Train err: 0.39416666666666667, Train loss: 1.3047558525775342  |Validation err: 0.391 , Validation loss:1.2812358386932858 
Total time elapsed: 182.76 seconds


In [ ]:
train(netalex, batch_size = 32, learning_rate = 0.001, num_epochs = 15)

Epoch 1: Train err: 0.8636666666666667, Train loss: 6.696951688604152  |Validation err: 0.8235 , Validation loss:2.198826481425573 
Epoch 2: Train err: 0.7846666666666666, Train loss: 2.0924417465291123  |Validation err: 0.7975 , Validation loss:2.510080871127901 
Epoch 3: Train err: 0.7626666666666667, Train loss: 2.0622327657456094  |Validation err: 0.7625 , Validation loss:2.4051466385523477 
Epoch 4: Train err: 0.751, Train loss: 2.019769406065028  |Validation err: 0.7625 , Validation loss:2.2512343421814935 
Epoch 5: Train err: 0.7235, Train loss: 1.9731543495299968  |Validation err: 0.7465 , Validation loss:2.336838175380041 
Epoch 6: Train err: 0.6973333333333334, Train loss: 1.9247440936717581  |Validation err: 0.7205 , Validation loss:2.2657625448136103 
Epoch 7: Train err: 0.6743333333333333, Train loss: 1.8526354774515679  |Validation err: 0.6925 , Validation loss:2.011690378189087 
Epoch 8: Train err: 0.6598333333333334, Train loss: 1.827751729082554  |Validation err: 0.723

In [ ]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 16, learning_rate = 0.0001, num_epochs = 20)

Epoch 1: Train err: 0.3865, Train loss: 1.1424561700922378  |Validation err: 0.2865671641791045 , Validation loss:0.9071825702512075 
Epoch 2: Train err: 0.197, Train loss: 0.6006449099550856  |Validation err: 0.3686567164179104 , Validation loss:1.1458514368250257 
Epoch 3: Train err: 0.11216666666666666, Train loss: 0.35121220500862343  |Validation err: 0.30895522388059704 , Validation loss:1.091131821038231 
Epoch 4: Train err: 0.07083333333333333, Train loss: 0.21470273650707083  |Validation err: 0.309452736318408 , Validation loss:1.3678297010206042 
Epoch 5: Train err: 0.0425, Train loss: 0.1261425785958133  |Validation err: 0.31840796019900497 , Validation loss:1.5226205663075523 
Epoch 6: Train err: 0.03866666666666667, Train loss: 0.11121613052772715  |Validation err: 0.32487562189054725 , Validation loss:1.636424587359504 
Epoch 7: Train err: 0.024333333333333332, Train loss: 0.06661355166517674  |Validation err: 0.3238805970149254 , Validation loss:1.791774418618944 
Epoch 8

In [ ]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 32, learning_rate = 0.0005, num_epochs = 20)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.torch/models/alexnet-owt-4df8aa71.pth
244418560it [00:04, 51536996.70it/s]


Epoch 1: Train err: 0.8931666666666667, Train loss: 2.3129352813071393  |Validation err: 0.900497512437811 , Validation loss:2.3032867908477783 
Epoch 2: Train err: 0.878, Train loss: 2.2840839675132263  |Validation err: 0.8766169154228856 , Validation loss:2.2816719819629 
Epoch 3: Train err: 0.8566666666666667, Train loss: 2.2538155393397554  |Validation err: 0.8671641791044776 , Validation loss:2.270226516420879 
Epoch 4: Train err: 0.8348333333333333, Train loss: 2.218134271337631  |Validation err: 0.836318407960199 , Validation loss:2.219892628609188 
Epoch 5: Train err: 0.8061666666666667, Train loss: 2.1489719976770116  |Validation err: 0.8238805970149253 , Validation loss:2.668194231532869 
Epoch 6: Train err: 0.7178333333333333, Train loss: 1.9751741493001898  |Validation err: 0.7079601990049751 , Validation loss:2.018897395285349 
Epoch 7: Train err: 0.601, Train loss: 1.6993810927614252  |Validation err: 0.6975124378109453 , Validation loss:2.4539715827457487 
Epoch 8: Train

In [ ]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 64, learning_rate = 0.0005, num_epochs = 20)

Epoch 1: Train err: 0.8835, Train loss: 2.320931708559077  |Validation err: 0.8791044776119403 , Validation loss:2.2924960075862826 
Epoch 2: Train err: 0.8613333333333333, Train loss: 2.2638895029717303  |Validation err: 0.8771144278606965 , Validation loss:2.285294078645252 
Epoch 3: Train err: 0.828, Train loss: 2.179639473874518  |Validation err: 0.818407960199005 , Validation loss:2.2222018563558184 
Epoch 4: Train err: 0.7263333333333334, Train loss: 1.9586204531344962  |Validation err: 0.6567164179104478 , Validation loss:1.811238551896716 
Epoch 5: Train err: 0.6023333333333334, Train loss: 1.652762827721048  |Validation err: 0.6343283582089553 , Validation loss:1.8037584414557806 
Epoch 6: Train err: 0.473, Train loss: 1.3626352485190047  |Validation err: 0.508955223880597 , Validation loss:1.6825155871255058 
Epoch 7: Train err: 0.3635, Train loss: 1.0854070890457073  |Validation err: 0.5084577114427861 , Validation loss:1.517351762642936 
Epoch 8: Train err: 0.30366666666666

In [ ]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 32, learning_rate = 0.00005, num_epochs = 20)

Epoch 1: Train err: 0.4196666666666667, Train loss: 1.2426844055348254  |Validation err: 0.2990049751243781 , Validation loss:0.920150447459448 
Epoch 2: Train err: 0.219, Train loss: 0.6540741188094971  |Validation err: 0.3567164179104478 , Validation loss:1.1191261514784798 
Epoch 3: Train err: 0.156, Train loss: 0.473246028765719  |Validation err: 0.2860696517412935 , Validation loss:0.926016146228427 
Epoch 4: Train err: 0.10583333333333333, Train loss: 0.3268153307602761  |Validation err: 0.3119402985074627 , Validation loss:1.1126495798428853 
Epoch 5: Train err: 0.06833333333333333, Train loss: 0.21071445062122446  |Validation err: 0.31144278606965176 , Validation loss:1.370934506257375 
Epoch 6: Train err: 0.0505, Train loss: 0.15983863476108998  |Validation err: 0.2716417910447761 , Validation loss:1.151049964957767 
Epoch 7: Train err: 0.031, Train loss: 0.09941372552767713  |Validation err: 0.2796019900497512 , Validation loss:1.213928859385233 
Epoch 8: Train err: 0.0281666

In [ ]:
netalex = AlexNet()
netalex.to(device)
train(netalex, batch_size = 64, learning_rate = 0.00005, num_epochs = 20)

Epoch 1: Train err: 0.41883333333333334, Train loss: 1.2419434785842896  |Validation err: 0.3034825870646766 , Validation loss:0.9177590764704204 
Epoch 2: Train err: 0.22016666666666668, Train loss: 0.657989907454937  |Validation err: 0.35124378109452736 , Validation loss:1.1080554543979584 
Epoch 3: Train err: 0.15733333333333333, Train loss: 0.47340591885942096  |Validation err: 0.291044776119403 , Validation loss:0.9185986149878729 
Epoch 4: Train err: 0.103, Train loss: 0.32332505919831867  |Validation err: 0.31044776119402984 , Validation loss:1.113845521022403 
Epoch 5: Train err: 0.0685, Train loss: 0.21185640697466565  |Validation err: 0.3054726368159204 , Validation loss:1.3256247895104545 
Epoch 6: Train err: 0.051, Train loss: 0.15742263427757203  |Validation err: 0.2845771144278607 , Validation loss:1.2137668506493644 
Epoch 7: Train err: 0.029833333333333333, Train loss: 0.09628963585388153  |Validation err: 0.2860696517412935 , Validation loss:1.2755623246942247 
Epoch 8

In [ ]:
netalex2 = AlexNet()
netalex2.to(device)
train(netalex2, batch_size = 32, learning_rate = 0.00001, num_epochs = 20)

Epoch 1: Train err: 0.5936666666666667, Train loss: 1.809698923471126  |Validation err: 0.41691542288557215 , Validation loss:1.226113047864702 
Epoch 2: Train err: 0.3368333333333333, Train loss: 1.013751449103051  |Validation err: 0.3497512437810945 , Validation loss:1.0828766529522245 
Epoch 3: Train err: 0.26616666666666666, Train loss: 0.8230058829835121  |Validation err: 0.345273631840796 , Validation loss:1.0834999680519104 
Epoch 4: Train err: 0.23733333333333334, Train loss: 0.7154985222410648  |Validation err: 0.3338308457711443 , Validation loss:1.059084074837821 
Epoch 5: Train err: 0.207, Train loss: 0.6225019794829348  |Validation err: 0.3174129353233831 , Validation loss:1.0316127169699896 
Epoch 6: Train err: 0.18166666666666667, Train loss: 0.5609518210304544  |Validation err: 0.3074626865671642 , Validation loss:1.0131385222313896 
Epoch 7: Train err: 0.17233333333333334, Train loss: 0.5021994854224489  |Validation err: 0.29701492537313434 , Validation loss:0.96660196

In [ ]:
netalex2 = AlexNet()
netalex2.to(device)
train(netalex2, batch_size = 32, learning_rate = 0.000005, num_epochs = 20)

Epoch 1: Train err: 0.7033333333333334, Train loss: 2.0962643889670676  |Validation err: 0.5358208955223881 , Validation loss:1.6299511988957722 
Epoch 2: Train err: 0.4156666666666667, Train loss: 1.342432473568206  |Validation err: 0.4223880597014925 , Validation loss:1.2120010058085124 
Epoch 3: Train err: 0.33916666666666667, Train loss: 1.0346959842012284  |Validation err: 0.3925373134328358 , Validation loss:1.2089395097323827 
Epoch 4: Train err: 0.29483333333333334, Train loss: 0.9154414857955689  |Validation err: 0.354228855721393 , Validation loss:1.0923599704863534 
Epoch 5: Train err: 0.27, Train loss: 0.8197406967903705  |Validation err: 0.33880597014925373 , Validation loss:1.0811677045292325 
Epoch 6: Train err: 0.25066666666666665, Train loss: 0.767062412931564  |Validation err: 0.3497512437810945 , Validation loss:1.0846690213869488 
Epoch 7: Train err: 0.23166666666666666, Train loss: 0.7002718274263625  |Validation err: 0.336318407960199 , Validation loss:1.087264839

# Test Accuracy-Alex Net
Report the test accuracy of your best model. How does the test accuracy compare to part 4(d)?

In [ ]:
def get_accuracy(model):
    data = testset
    total = 0
    correct =0
    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=32):
        features = myfeature_model(imgs)
        output = net(features) # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
get_accuracy(net)

# ResNet18 

**FREEZE LAYERS + LEARNING RATE SCHEDULER**

In [55]:
def trainResNetnew(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
   
    
    from torchvision import transforms
    from torch.optim import lr_scheduler
    # Set up a transform that scales and crops an image so it has the dimensions
    # of the input layer of alexnet
    scale_crop = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224)
    ])
  
    # The normalization that was applied to the data when alexnet was trained
    normalize = transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
    )

    # Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
    preprocess = transforms.Compose([
      scale_crop,
      transforms.ToTensor(),
      normalize
    ])
    trainset = torchvision.datasets.ImageFolder(root = 'ig/train_more_food',
                                                 transform=preprocess)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='ig/val_more_food',
                                                 transform=preprocess)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'ig/test_more_food',
                                            transform=preprocess)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)
    
    
    torch.manual_seed(1000)

    optimizer = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)
    print("[Creating Learning rate scheduler...]")
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    
    criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.Adam(net.fc.parameters(), lr=learning_rate, weight_decay=1e-5)
   
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
        exp_lr_scheduler.step()
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            net.train() #######################################################
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
                      
            optimizer.zero_grad()
            outputs = net(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
           
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        net.eval() ###############################################3
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name("RESNETNEW", batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
        
          

In [56]:
model_conv = torchvision.models.resnet18(pretrained=True).to(device)
#num_ftrs = model_conv.fc.in_features
#model_conv.fc = nn.Linear(num_ftrs, 20).to(device)


In [57]:
##Code from: https://medium.com/@14prakash/almost-any-image-classification-problem-using-pytorch-i-am-in-love-with-pytorch-26c7aa979ec4

## Lets freeze the first few layers. This is done in two stages 
# Stage-1 Freezing all the layers 
from torch.optim import lr_scheduler
for i, param in model_conv.named_parameters():
    param.requires_grad = False

# Since imagenet as 1000 classes , We need to change our last layer according to the number of classes we have,
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 20)

# Stage-2 , Freeze all the layers till "Conv2d_4a_3*3"
ct = []
for name, child in model_conv.named_children():
    if "Conv2d_4a_3x3" in ct:
        for params in child.parameters():
            params.requires_grad = True
    ct.append(name)


In [58]:
model_conv = nn.DataParallel(model_conv, device_ids=[0, 1])
model_conv = model_conv.to(device)


In [37]:
trainResNetnew(model_conv, batch_size=128, learning_rate=0.001, num_epochs=20)

[Creating Learning rate scheduler...]
Epoch 1: Train err: 0.72425, Train loss: 2.63157956143643  |Validation err: 0.4785 , Validation loss:2.113164186477661 
Epoch 2: Train err: 0.39775, Train loss: 1.8542349668259317  |Validation err: 0.34575 , Validation loss:1.5982104651629925 
Epoch 3: Train err: 0.31666666666666665, Train loss: 1.4835851306610919  |Validation err: 0.30375 , Validation loss:1.3485260382294655 
Epoch 4: Train err: 0.2925833333333333, Train loss: 1.2892775002946244  |Validation err: 0.28275 , Validation loss:1.2002365235239267 
Epoch 5: Train err: 0.2773333333333333, Train loss: 1.16473421898294  |Validation err: 0.27475 , Validation loss:1.1140891183167696 
Epoch 6: Train err: 0.2669166666666667, Train loss: 1.0889027828865863  |Validation err: 0.2605 , Validation loss:1.0525336470454931 
Epoch 7: Train err: 0.25658333333333333, Train loss: 1.027977055691658  |Validation err: 0.2575 , Validation loss:1.0065363999456167 
Epoch 8: Train err: 0.25183333333333335, Train

KeyboardInterrupt: 

In [39]:
trainResNetnew(model_conv, batch_size=256, learning_rate=0.001, num_epochs=20)

[Creating Learning rate scheduler...]
Epoch 1: Train err: 0.24083333333333334, Train loss: 0.9401577977423973  |Validation err: 0.248 , Validation loss:0.9585955888032913 
Epoch 2: Train err: 0.23983333333333334, Train loss: 0.9259341042092506  |Validation err: 0.24675 , Validation loss:0.948080588132143 
Epoch 3: Train err: 0.236, Train loss: 0.9099200900564802  |Validation err: 0.24375 , Validation loss:0.9350448958575726 


Exception ignored in: <function _DataLoaderIter.__del__ at 0x7fc32bb45840>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch 4: Train err: 0.23308333333333334, Train loss: 0.8963869556467584  |Validation err: 0.2415 , Validation loss:0.9245823845267296 
Epoch 5: Train err: 0.23125, Train loss: 0.8810925458339934  |Validation err: 0.23825 , Validation loss:0.9046325832605362 
Epoch 6: Train err: 0.23141666666666666, Train loss: 0.8730097996427658  |Validation err: 0.24025 , Validation loss:0.8998244181275368 
Epoch 7: Train err: 0.22866666666666666, Train loss: 0.8603315289984358  |Validation err: 0.23825 , Validation loss:0.8881871588528156 
Epoch 8: Train err: 0.22683333333333333, Train loss: 0.8527368877796416  |Validation err: 0.2405 , Validation loss:0.8935535699129105 
Epoch 9: Train err: 0.22858333333333333, Train loss: 0.8506573387917052  |Validation err: 0.23975 , Validation loss:0.8898581750690937 
Epoch 10: Train err: 0.22366666666666668, Train loss: 0.8467783661598854  |Validation err: 0.239 , Validation loss:0.890851940959692 
Epoch 11: Train err: 0.22508333333333333, Train loss: 0.84529807

In [54]:
trainResNetnew(model_conv, batch_size=64, learning_rate=0.0001, num_epochs=20)

[Creating Learning rate scheduler...]
Epoch 1: Train err: 0.5885833333333333, Train loss: 2.2864720542380153  |Validation err: 0.366 , Validation loss:1.6232504598678104 
Epoch 2: Train err: 0.32075, Train loss: 1.431323174466478  |Validation err: 0.2905 , Validation loss:1.2172306416526673 
Epoch 3: Train err: 0.2813333333333333, Train loss: 1.1648825787483377  |Validation err: 0.27125 , Validation loss:1.0705701481728327 
Epoch 4: Train err: 0.2638333333333333, Train loss: 1.0400146585195622  |Validation err: 0.25775 , Validation loss:0.9707410874820891 
Epoch 5: Train err: 0.25508333333333333, Train loss: 0.9670079410710233  |Validation err: 0.25325 , Validation loss:0.9218486169027904 
Epoch 6: Train err: 0.24841666666666667, Train loss: 0.9164995123097237  |Validation err: 0.24425 , Validation loss:0.8839426296097892 
Epoch 7: Train err: 0.23775, Train loss: 0.8794192056706611  |Validation err: 0.236 , Validation loss:0.8593225971100822 
Epoch 8: Train err: 0.2315, Train loss: 0.8

In [60]:
trainResNetnew(model_conv, batch_size=128, learning_rate=0.001, num_epochs=20)

[Creating Learning rate scheduler...]
Epoch 1: Train err: 0.2515, Train loss: 0.9755658475642509  |Validation err: 0.25525 , Validation loss:0.965672891587019 
Epoch 2: Train err: 0.2455, Train loss: 0.9412747589831657  |Validation err: 0.24825 , Validation loss:0.9333129059523344 
Epoch 3: Train err: 0.23891666666666667, Train loss: 0.9105711896368798  |Validation err: 0.24675 , Validation loss:0.9135573450475931 
Epoch 4: Train err: 0.237, Train loss: 0.8877841589298654  |Validation err: 0.2435 , Validation loss:0.8881979025900364 
Epoch 5: Train err: 0.23366666666666666, Train loss: 0.8654963558024549  |Validation err: 0.24325 , Validation loss:0.8797290846705437 
Epoch 6: Train err: 0.23075, Train loss: 0.8493758674631727  |Validation err: 0.238 , Validation loss:0.8633231837302446 
Epoch 7: Train err: 0.22625, Train loss: 0.8316304531503231  |Validation err: 0.23675 , Validation loss:0.8556522950530052 
Epoch 8: Train err: 0.22516666666666665, Train loss: 0.8182203776024758  |Vali

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 20).to(device)
model_conv.to(device)
model_path = get_model_name("RESNET", batch_size=128, learning_rate=0.0001, epoch=29)
state = torch.load(model_path)
model_conv.load_state_dict(state)
evaluate(model_conv, test_loader, nn.CrossEntropyLoss())

(0.243, 0.8824597001075745)

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 20).to(device)
trainResNet(model_conv, batch_size=128, learning_rate=0.01, num_epochs=30)

Epoch 1: Train err: 0.39908485856905157, Train loss: 1.536391649474489  |Validation err: 0.2715 , Validation loss:0.9300655294209719 
Epoch 2: Train err: 0.24242928452579035, Train loss: 0.7993874010887552  |Validation err: 0.24175 , Validation loss:0.8212614338845015 
Epoch 3: Train err: 0.21647254575707156, Train loss: 0.7118265093006986  |Validation err: 0.242 , Validation loss:0.8290862459689379 
Epoch 4: Train err: 0.21264559068219635, Train loss: 0.6827567175347754  |Validation err: 0.25575 , Validation loss:0.8893328811973333 
Epoch 5: Train err: 0.2103161397670549, Train loss: 0.6960205194797922  |Validation err: 0.241 , Validation loss:0.8453835360705853 
Epoch 6: Train err: 0.19767054908485857, Train loss: 0.6480449022764855  |Validation err: 0.2475 , Validation loss:0.8726602476090193 
Epoch 7: Train err: 0.19176372712146422, Train loss: 0.6257810846288153  |Validation err: 0.259 , Validation loss:0.9814035221934319 
Epoch 8: Train err: 0.18885191347753744, Train loss: 0.620

In [55]:
model_conv = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 20).to(device)
trainResNet(model_conv, batch_size= 64, learning_rate=0.001, num_epochs=50)

Epoch 1: Train err: 0.4190833333333333, Train loss: 1.6637391020008858  |Validation err: 0.28175 , Validation loss:1.1033667969325232 
Epoch 2: Train err: 0.2614166666666667, Train loss: 0.9669207914712581  |Validation err: 0.25175 , Validation loss:0.8986421322065686 
Epoch 3: Train err: 0.23383333333333334, Train loss: 0.8246071782517941  |Validation err: 0.2365 , Validation loss:0.8361362767597985 
Epoch 4: Train err: 0.21791666666666668, Train loss: 0.752167339654679  |Validation err: 0.242 , Validation loss:0.8110082717168898 
Epoch 5: Train err: 0.20683333333333334, Train loss: 0.7075566733454136  |Validation err: 0.23975 , Validation loss:0.7911841055703541 
Epoch 6: Train err: 0.20108333333333334, Train loss: 0.680377427250781  |Validation err: 0.23325 , Validation loss:0.787389695171326 
Epoch 7: Train err: 0.1935, Train loss: 0.6540768019696499  |Validation err: 0.22575 , Validation loss:0.7578528816737826 
Epoch 8: Train err: 0.18633333333333332, Train loss: 0.63446460474044

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

# Modified Resnet

In [14]:
model_resmod = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_resmod.fc.in_features
model_resmod.fc = nn.Sequential(nn.Linear(num_ftrs, 216).to(device),nn.ReLU().to(device),
                               nn.Linear(216,128).to(device),nn.Dropout(0.5).to(device),nn.ReLU(),
,                                nn.Linear(128,20).to(device))
#print(model_resmod)
trainResNetDA(model_resmod, batch_size= 64, learning_rate=0.0001, num_epochs=30)

Epoch 1: Train err: 0.6362083333333334, Train loss: 2.4103850440979  |Validation err: 0.3955 , Validation loss:1.6493736865028503 
Total time elapsed: 197.47 seconds
Epoch 2: Train err: 0.3877083333333333, Train loss: 1.435254467010498  |Validation err: 0.32025 , Validation loss:1.153655659584772 
Total time elapsed: 363.61 seconds
Epoch 3: Train err: 0.34025, Train loss: 1.1759901294708253  |Validation err: 0.29925 , Validation loss:1.0359827298966666 
Total time elapsed: 530.30 seconds
Epoch 4: Train err: 0.3162083333333333, Train loss: 1.0729954274495443  |Validation err: 0.2845 , Validation loss:0.9626742431095668 
Total time elapsed: 696.20 seconds
Epoch 5: Train err: 0.30275, Train loss: 1.0231116800308226  |Validation err: 0.26925 , Validation loss:0.9241339852885594 
Total time elapsed: 862.93 seconds
Epoch 6: Train err: 0.296375, Train loss: 0.9874955123265584  |Validation err: 0.27 , Validation loss:0.8897495799594455 
Total time elapsed: 1029.16 seconds
Epoch 7: Train err: 0

In [15]:
model_resmod = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_resmod.fc.in_features
model_resmod.fc = nn.Sequential(nn.Linear(num_ftrs, 216).to(device),nn.ReLU().to(device),
                               nn.Linear(216,128).to(device),nn.Dropout(0.5).to(device),nn.ReLU(),
                                nn.Linear(128,20).to(device))
#print(model_resmod)
trainResNetDA(model_resmod, batch_size= 64, learning_rate=0.0005, num_epochs=30)

Epoch 1: Train err: 0.45945833333333336, Train loss: 1.6057487428983053  |Validation err: 0.3065 , Validation loss:1.0178756439496601 
Total time elapsed: 165.14 seconds
Epoch 2: Train err: 0.31075, Train loss: 1.045701271057129  |Validation err: 0.28075 , Validation loss:0.9289043119975499 
Total time elapsed: 331.90 seconds
Epoch 3: Train err: 0.28958333333333336, Train loss: 0.9665951420466106  |Validation err: 0.26975 , Validation loss:0.9021504578136262 
Total time elapsed: 497.91 seconds
Epoch 4: Train err: 0.27516666666666667, Train loss: 0.9121582752068838  |Validation err: 0.257 , Validation loss:0.8486317831372457 
Total time elapsed: 664.08 seconds
Epoch 5: Train err: 0.26466666666666666, Train loss: 0.8784831972122192  |Validation err: 0.25275 , Validation loss:0.8538888555670542 
Total time elapsed: 829.22 seconds
Epoch 6: Train err: 0.25366666666666665, Train loss: 0.8446724543571472  |Validation err: 0.24725 , Validation loss:0.8270000309225113 
Total time elapsed: 995.5

In [20]:
model_resmod = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_resmod.fc.in_features
model_resmod.fc = nn.Sequential(nn.Linear(num_ftrs, 216).to(device),nn.ReLU().to(device),
                               nn.Linear(216,128).to(device),nn.Dropout(0.5).to(device),nn.ReLU(),
                                nn.Linear(128,20).to(device))
#print(model_resmod)
trainResNet(model_resmod, batch_size= 64, learning_rate=0.00005, num_epochs=30)

Epoch 1: Train err: 0.83725, Train loss: 2.8770094272938183  |Validation err: 0.68375 , Validation loss:2.711021544441344 
Total time elapsed: 89.88 seconds
Epoch 2: Train err: 0.5739166666666666, Train loss: 2.4888841839546854  |Validation err: 0.50425 , Validation loss:2.244612799750434 
Total time elapsed: 181.25 seconds
Epoch 3: Train err: 0.453, Train loss: 2.018533447321425  |Validation err: 0.417 , Validation loss:1.8162292469115484 
Total time elapsed: 272.73 seconds
Epoch 4: Train err: 0.39158333333333334, Train loss: 1.6644351767732741  |Validation err: 0.3845 , Validation loss:1.5487727759376404 
Total time elapsed: 363.96 seconds
Epoch 5: Train err: 0.36441666666666667, Train loss: 1.4300847040845992  |Validation err: 0.36625 , Validation loss:1.3757558304166038 
Total time elapsed: 454.42 seconds
Epoch 6: Train err: 0.34358333333333335, Train loss: 1.295385741807045  |Validation err: 0.3405 , Validation loss:1.2574809638280717 
Total time elapsed: 544.96 seconds
Epoch 7: T

In [21]:
model_resmod = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_resmod.fc.in_features
model_resmod.fc = nn.Sequential(nn.Linear(num_ftrs, 216).to(device),nn.ReLU().to(device),
                               nn.Linear(216,128).to(device),nn.Dropout(0.5).to(device),nn.ReLU(),
                                nn.Linear(128,20).to(device))
#print(model_resmod)
trainResNet(model_resmod, batch_size= 64, learning_rate=0.0001, num_epochs=30)

Epoch 1: Train err: 0.72975, Train loss: 2.7036956802327583  |Validation err: 0.51825 , Validation loss:2.281234934216454 
Total time elapsed: 90.36 seconds
Epoch 2: Train err: 0.4374166666666667, Train loss: 1.8774740905203717  |Validation err: 0.39425 , Validation loss:1.5594886560288688 
Total time elapsed: 180.58 seconds
Epoch 3: Train err: 0.36125, Train loss: 1.3882280721309337  |Validation err: 0.34375 , Validation loss:1.2676766770226615 
Total time elapsed: 271.57 seconds
Epoch 4: Train err: 0.3259166666666667, Train loss: 1.1789664280541399  |Validation err: 0.31925 , Validation loss:1.1325273778703477 
Total time elapsed: 361.63 seconds
Epoch 5: Train err: 0.30366666666666664, Train loss: 1.0534963227332907  |Validation err: 0.31325 , Validation loss:1.060103484562465 
Total time elapsed: 452.56 seconds
Epoch 6: Train err: 0.2935, Train loss: 0.9952764872540819  |Validation err: 0.289 , Validation loss:1.000165637523409 
Total time elapsed: 544.26 seconds
Epoch 7: Train err:

In [9]:
model_resmod = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_resmod.fc.in_features
model_resmod.fc = nn.Sequential(nn.Linear(num_ftrs, 216).to(device),nn.ReLU().to(device),
                               nn.Linear(216,128).to(device),nn.Dropout(0.5).to(device),nn.ReLU(),
                                nn.Linear(128,20).to(device))
#print(model_resmod)
trainResNet(model_resmod, batch_size= 64, learning_rate=0.0005, num_epochs=30)

Epoch 1: Train err: 0.5026666666666667, Train loss: 1.8296967342178871  |Validation err: 0.338 , Validation loss:1.1457969811227586 
Total time elapsed: 120.10 seconds
Epoch 2: Train err: 0.31733333333333336, Train loss: 1.0567598025849525  |Validation err: 0.31025 , Validation loss:1.0112297393026806 
Total time elapsed: 213.07 seconds
Epoch 3: Train err: 0.274, Train loss: 0.9138582894142638  |Validation err: 0.28625 , Validation loss:0.9523319015427242 
Total time elapsed: 303.69 seconds
Epoch 4: Train err: 0.24858333333333332, Train loss: 0.835197775129308  |Validation err: 0.2695 , Validation loss:0.9064487010713608 
Total time elapsed: 394.43 seconds
Epoch 5: Train err: 0.23841666666666667, Train loss: 0.7809689485646308  |Validation err: 0.27175 , Validation loss:0.89109867811203 
Total time elapsed: 484.87 seconds
Epoch 6: Train err: 0.23016666666666666, Train loss: 0.7588379240416466  |Validation err: 0.259 , Validation loss:0.8772706871940976 
Total time elapsed: 574.65 secon

In [10]:
model_resmod = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_resmod.fc.in_features
model_resmod.fc = nn.Sequential(nn.Linear(num_ftrs, 216).to(device),nn.ReLU().to(device),
                               nn.Linear(216,128).to(device),nn.Dropout(0.5).to(device),nn.ReLU(),
                                nn.Linear(128,20).to(device))
#print(model_resmod)
trainResNet(model_resmod, batch_size= 128, learning_rate=0.0001, num_epochs=30)

Epoch 1: Train err: 0.8006666666666666, Train loss: 2.8293524853726653  |Validation err: 0.6445 , Validation loss:2.602300748229027 
Total time elapsed: 90.60 seconds
Epoch 2: Train err: 0.5210833333333333, Train loss: 2.2931180215896445  |Validation err: 0.45575 , Validation loss:1.985670655965805 
Total time elapsed: 182.07 seconds
Epoch 3: Train err: 0.40941666666666665, Train loss: 1.7400200924974807  |Validation err: 0.38875 , Validation loss:1.5587487891316414 
Total time elapsed: 271.31 seconds
Epoch 4: Train err: 0.35991666666666666, Train loss: 1.4129762066171525  |Validation err: 0.3515 , Validation loss:1.320157129317522 
Total time elapsed: 362.76 seconds
Epoch 5: Train err: 0.33, Train loss: 1.227053221235884  |Validation err: 0.332 , Validation loss:1.1917029991745949 
Total time elapsed: 453.69 seconds
Epoch 6: Train err: 0.31825, Train loss: 1.1217031092085736  |Validation err: 0.312 , Validation loss:1.1080899387598038 
Total time elapsed: 544.48 seconds
Epoch 7: Train

In [ ]:
model_resmod = torchvision.models.resnet18(pretrained=True).to(device)
num_ftrs = model_resmod.fc.in_features
model_resmod.fc = nn.Sequential(nn.Linear(num_ftrs, 216).to(device),nn.ReLU().to(device),
                               nn.Linear(216,128).to(device),nn.Dropout(0.5).to(device),nn.ReLU(),
                                nn.Linear(128,20).to(device))
#print(model_resmod)
trainResNet(model_resmod, batch_size= 128, learning_rate=0.00001, num_epochs=30)

Epoch 1: Train err: 0.9435, Train loss: 2.996792098309131  |Validation err: 0.9345 , Validation loss:2.9902035742998123 
Total time elapsed: 113.82 seconds
Epoch 2: Train err: 0.9305833333333333, Train loss: 2.980627658519339  |Validation err: 0.9175 , Validation loss:2.9709274917840958 
Total time elapsed: 206.23 seconds
Epoch 3: Train err: 0.89925, Train loss: 2.9599617795741304  |Validation err: 0.88825 , Validation loss:2.946491815149784 
Total time elapsed: 296.02 seconds
Epoch 4: Train err: 0.87525, Train loss: 2.9392296786003924  |Validation err: 0.8565 , Validation loss:2.925736367702484 
Total time elapsed: 387.07 seconds
Epoch 5: Train err: 0.8485, Train loss: 2.9149233204253178  |Validation err: 0.82725 , Validation loss:2.8978857323527336 
Total time elapsed: 479.95 seconds
Epoch 6: Train err: 0.8155833333333333, Train loss: 2.885659085943344  |Validation err: 0.80425 , Validation loss:2.8678610250353813 
Total time elapsed: 569.98 seconds
Epoch 7: Train err: 0.789166666666